In [373]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [497]:
# config = {
#     'user': 'root',
#     'password': 'Hh12H8y2ppBAq5Cj',
#     'host': '34.65.175.109',
#     'client_flags': [ClientFlag.SSL],
#     'ssl_ca': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/server-ca.pem',
#     'ssl_cert': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-cert.pem',
#     'ssl_key': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-key.pem',
#     'database': 'data'
# }

config = {
    'user': 'root',
    'password': 'Hh12H8y2ppBAq5Cj',
    'host': 'localhost',
    'database': 'data'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

# Party
## Main Tables

In [404]:
cursor.execute("""CREATE TABLE Parties(id INTEGER,
                        case_id NUMERIC(19) NOT NULL,
                        party_number INTEGER NOT NULL,
                        financial_responsibility INTEGER,
                        party_age INTEGER,
                        party_sex INTEGER,
                        at_fault BIT NOT NULL,
                        cellphone_use INTEGER,
                        hazardous_materials CHAR(1),
                        movement_preceding_collision INTEGER,
                        other_associate_factor_1 INTEGER,
                        other_associate_factor_2 INTEGER,
                        party_drug_physical INTEGER,
                        party_safety_equipment_1 INTEGER,
                        party_safety_equipment_2 INTEGER,
                        party_sobriety INTEGER,
                        PRIMARY KEY(id),
                        FOREIGN KEY(financial_responsibility) REFERENCES FinancialResponsibility(id),
                        FOREIGN KEY(party_sex) REFERENCES PersonSex(id),
                        FOREIGN KEY(cellphone_use) REFERENCES CellphoneUse(id),
                        FOREIGN KEY(movement_preceding_collision) REFERENCES MovementPrecedingCollision(id),
                        FOREIGN KEY(other_associate_factor_1) REFERENCES OtherAssociatedFactors(id),
                        FOREIGN KEY(other_associate_factor_2) REFERENCES OtherAssociatedFactors(id),
                        FOREIGN KEY(party_drug_physical) REFERENCES PartyDrugPhysical(id),
                        FOREIGN KEY(party_safety_equipment_1) REFERENCES PartySafetyEquipement(id),
                        FOREIGN KEY(party_safety_equipment_2) REFERENCES PartySafetyEquipement(id),
                        FOREIGN KEY(party_sobriety) REFERENCES PartySobriety(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [405]:
cursor.execute("""CREATE TABLE Vehicles(party_id INTEGER NOT NULL,
                        school_bus_related BIT,
                        statewide_vehicle_type INTEGER,
                        vehicle_make INTEGER,
                        vehicle_year INTEGER,
                        FOREIGN KEY(statewide_vehicle_type) REFERENCES StatewideVehiculeType(id),
                        FOREIGN KEY(vehicle_make) REFERENCES VehiculeMake(id),
                        FOREIGN KEY(party_id) REFERENCES Parties(id))""")

## Satelite Enum Tables

In [393]:
cursor.execute("CREATE TABLE OtherAssociatedFactors (id INTEGER AUTO_INCREMENT,"
    "description CHAR(1) NOT NULL UNIQUE,"
    "CHECK (description BETWEEN 'A' AND 'Z'),"
    "PRIMARY KEY(id))")

In [394]:
cursor.execute("""CREATE TABLE CellphoneUse(id INTEGER AUTO_INCREMENT,
			   description CHAR(1) NOT NULL UNIQUE,
                CHECK (description BETWEEN 'B' AND 'D'),	
			   PRIMARY KEY(id))""")
# messy data: 1 -> B, 2 -> C, 3 -> D

In [395]:
cursor.execute("""CREATE TABLE FinancialResponsibility(id INTEGER AUTO_INCREMENT,
			   	      description CHAR(1) NOT NULL UNIQUE,
                                      CHECK (description IN ('N', 'Y', 'O', 'E', '-')),	
			              PRIMARY KEY(id))""")

# clean data 

In [396]:
cursor.execute("""CREATE TABLE MovementPrecedingCollision(id INTEGER AUTO_INCREMENT,
					 description VARCHAR(100) NOT NULL UNIQUE,
					 PRIMARY KEY(id))""")
# seems clean, but cannot check every possibility as it's varchar.

In [397]:
cursor.execute("""CREATE TABLE PartyDrugPhysical(id INTEGER AUTO_INCREMENT,
			       description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('E', 'F', 'H', 'I')),
			       PRIMARY KEY(id))""")

# messy: letter G most prevalent: interpret as n/a ?

In [398]:
cursor.execute("""CREATE TABLE PartySafetyEquipement(id INTEGER AUTO_INCREMENT,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK (description BETWEEN 'A' AND 'Y'),
				   PRIMARY KEY(id))""")
# clean as far as i can tell

The following table holds for `VictimSex` and `PartySex`

In [399]:
cursor.execute("""CREATE TABLE PersonSex(id INTEGER AUTO_INCREMENT,
		       description VARCHAR(6) NOT NULL UNIQUE,
                       CHECK (description in ('male', 'female')),
		       PRIMARY KEY(id))""")
# clean af, for PartySex and VictimSex

In [400]:
cursor.execute("""CREATE TABLE PartySobriety(id INTEGER AUTO_INCREMENT,
			    description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('A', 'B', 'C', 'D', 'G', 'H')),
			    PRIMARY KEY(id))""")
# clean af

In [401]:
cursor.execute("""CREATE TABLE PartyType(id INTEGER AUTO_INCREMENT,
			description VARCHAR(15) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")

# clean

In [402]:
cursor.execute("""CREATE TABLE StatewideVehiculeType(id INTEGER AUTO_INCREMENT,
				    description VARCHAR(35) NOT NULL UNIQUE,
				    PRIMARY KEY(id))""")
# clean as far as i can tell

In [403]:
cursor.execute("""CREATE TABLE VehiculeMake(id INTEGER AUTO_INCREMENT,
			   description VARCHAR(28) NOT NULL,
			   PRIMARY KEY(id))""")

# clean

# Collision
## Main Tables

In [388]:
cursor.execute("""CREATE TABLE Collisions(case_id NUMERIC(19), 
                        collision_date DATE NOT NULL,
                        collision_time TIME,
                        type_of_collision INTEGER,
                        collision_severity INTEGER NOT NULL,
                        hit_and_run INTEGER NOT NULL,
                        tow_away BIT,
                        FOREIGN KEY(type_of_collision) REFERENCES TypeOfCollision(id), 
                        FOREIGN KEY(collision_severity) REFERENCES CollisionSeverity(id), 
                        FOREIGN KEY(hit_and_run) REFERENCES HitAndRun(id),                      
                        PRIMARY KEY(case_id))""")

In [389]:
cursor.execute("""CREATE TABLE Pcfs(case_id NUMERIC(19) NOT NULL,
                    pcf_violation INTEGER,
                    pcf_violation_category INTEGER,
                    pcf_violation_subsection CHAR(1),
                    FOREIGN KEY(pcf_violation_category) REFERENCES PcfViolationCategory(id),
                    FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")
# pcf_violation_subscection has weird values but we cannot infer
# that they are wrong.

In [390]:
cursor.execute("""CREATE TABLE Locations(case_id NUMERIC(19) NOT NULL,
                        population INTEGER,
                        county_city_location INTEGER NOT NULL,
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [391]:
cursor.execute("""CREATE TABLE Factors(case_id NUMERIC(19) NOT NULL,
                        location_type INTEGER,
                        lighting INTEGER,
                        road_condition_1 INTEGER,
                        road_condition_2 INTEGER,
                        road_surface INTEGER,
                        weather_1 INTEGER,
                        weather_2 INTEGER,
                        FOREIGN KEY(location_type) REFERENCES LocationType(id),
                        FOREIGN KEY(lighting) REFERENCES Lighting(id),
                        FOREIGN KEY(road_condition_1) REFERENCES RoadCondition(id),
                        FOREIGN KEY(road_condition_2) REFERENCES RoadCondition(id),
                        FOREIGN KEY(road_surface) REFERENCES RoadSurface(id),
                        FOREIGN KEY(weather_1) REFERENCES Weather(id),
                        FOREIGN KEY(weather_2) REFERENCES Weather(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [392]:
cursor.execute("""CREATE TABLE Cases(case_id NUMERIC(19) NOT NULL,
                    process_date DATE NOT NULL,
                    officer_id VARCHAR(8),
                    jurisdiction INTEGER,
                    FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")
# max officer_id length is 8. As some include letters, better make it VARCHAR.

## Satelite Enum Tables

In [378]:
cursor.execute("""CREATE TABLE CollisionSeverity(id INTEGER AUTO_INCREMENT,
			       description VARCHAR(25) NOT NULL UNIQUE,
			       PRIMARY KEY(id))""")
# clean

In [379]:
cursor.execute("""CREATE TABLE HitAndRun(id INTEGER AUTO_INCREMENT,
			description VARCHAR(20) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")
# One dirty value: D -> NaN 

In [380]:
cursor.execute("""CREATE TABLE Lighting(id INTEGER AUTO_INCREMENT,
		       description VARCHAR(39) NOT NULL UNIQUE,
		       PRIMARY KEY(id))""")
# Clean

In [381]:
cursor.execute("""CREATE TABLE LocationType(id INTEGER AUTO_INCREMENT,
			   description VARCHAR(20) NOT NULL UNIQUE,
			   PRIMARY KEY(id))""")
# clean

In [382]:
cursor.execute("""CREATE TABLE PcfViolationCategory(id INTEGER AUTO_INCREMENT,
				   description VARCHAR(70) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")

# One dirty value: 21804 -> NaN

In [383]:
cursor.execute("""CREATE TABLE PrimaryCollisionFactor(id INTEGER AUTO_INCREMENT, 
				     description VARCHAR(25) NOT NULL UNIQUE,
				     PRIMARY KEY(id))""")
# clean

In [384]:
cursor.execute("""CREATE TABLE RoadCondition(id INTEGER AUTO_INCREMENT,
			    description VARCHAR(20) NOT NULL UNIQUE,
			    PRIMARY KEY(id))""")
# clean

In [385]:
cursor.execute("""CREATE TABLE RoadSurface(id INTEGER AUTO_INCREMENT,
			  description VARCHAR(15) NOT NULL UNIQUE,
			  PRIMARY KEY(id))""")
# one dirty value: H -> NaN

In [386]:
cursor.execute("""CREATE TABLE TypeOfCollision(id INTEGER AUTO_INCREMENT,
			      description VARCHAR(15) NOT NULL UNIQUE,
			      PRIMARY KEY(id))""")
# clean

In [387]:
cursor.execute("""CREATE TABLE Weather(id INTEGER AUTO_INCREMENT,
		      description VARCHAR(10) NOT NULL UNIQUE,
		      PRIMARY KEY(id))""")
# clean

# Victim
## Main Table

In [408]:
cursor.execute("""CREATE TABLE Victims(id INTEGER,
                        case_id NUMERIC(19) NOT NULL,
                        party_number INTEGER NOT NULL,
                        victim_age INTEGER,
                        victim_sex INTEGER,
                        victim_degree_of_injury INTEGER,
                        victim_ejected INTEGER,
                        victim_role INTEGER NOT NULL,
                        victim_safety_equipment_1 INTEGER,
                        victim_safety_equipment_2 INTEGER,
                        victim_seating_position INTEGER,
                        PRIMARY KEY(id),
                        FOREIGN KEY(victim_sex) REFERENCES PersonSex(id),
                        FOREIGN KEY(victim_safety_equipment_1) REFERENCES VictimSafetyEquipment(id),
                        FOREIGN KEY(victim_safety_equipment_2) REFERENCES VictimSafetyEquipment(id),
                        FOREIGN KEY(victim_degree_of_injury) REFERENCES VictimDegreeOfInjury(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

## Satelite Table

In [406]:
cursor.execute("""CREATE TABLE VictimSafetyEquipment(id INTEGER AUTO_INCREMENT,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK(description BETWEEN 'A' AND 'Z'),
				   PRIMARY KEY(id))""")

In [407]:
cursor.execute("""CREATE TABLE VictimDegreeOfInjury(id INTEGER AUTO_INCREMENT,
				   description VARCHAR(30) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")
# one drity value: 7 -> NaN

# Data import
## Alterations to table definitions during data import

Only use `SET FOREIGN_KEY_CHECKS` during definition of tables, *not in production*

In [205]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE CollisionSeverity MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE HitAndRun MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE Lighting MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE LocationType MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PrimaryCollisionFactor MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE RoadCondition MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE RoadSurface MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE TypeOfCollision MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE Weather MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PcfViolationCategory MODIFY id INTEGER AUTO_INCREMENT")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [63]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE CellphoneUse MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE FinancialResponsibility MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE MovementPrecedingCollision MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE OtherAssociatedFactors MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartyDrugPhysical MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartySafetyEquipement MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PersonSex MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartySobriety MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartyType MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE StatewideVehiculeType MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE VehiculeMake MODIFY id INTEGER AUTO_INCREMENT")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [64]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE VictimSafetyEquipment MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE VictimDegreeOfInjury MODIFY id INTEGER AUTO_INCREMENT")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [238]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE Lighting MODIFY COLUMN description VARCHAR(39)")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [563]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE StatewideVehiculeType MODIFY COLUMN description VARCHAR(35)")
cursor.execute("ALTER TABLE VehiculeMake MODIFY COLUMN description VARCHAR(28)")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [493]:
cursor.execute("ALTER TABLE FinancialResponsability RENAME FinancialResponsibility")

In [501]:
cursor.execute("ALTER TABLE Parties RENAME COLUMN finanicial_responsibility TO financial_responsibility")

In [502]:
cursor.execute("SHOW CREATE TABLE Parties")
res = cursor.fetchall()[0]
for x in res:
    print(x)

Parties
CREATE TABLE `Parties` (
  `id` int NOT NULL,
  `case_id` decimal(19,0) NOT NULL,
  `party_number` int NOT NULL,
  `financial_responsibility` int DEFAULT NULL,
  `party_age` int DEFAULT NULL,
  `party_sex` int DEFAULT NULL,
  `at_fault` bit(1) NOT NULL,
  `cellphone_use` int DEFAULT NULL,
  `hazardous_materials` char(1) DEFAULT NULL,
  `movement_preceding_collision` int DEFAULT NULL,
  `other_associate_factor_1` int DEFAULT NULL,
  `other_associate_factor_2` int DEFAULT NULL,
  `party_drug_physical` int DEFAULT NULL,
  `party_safety_equipment_1` int DEFAULT NULL,
  `party_safety_equipment_2` int DEFAULT NULL,
  `party_sobriety` int DEFAULT NULL,
  PRIMARY KEY (`id`),
  KEY `finanicial_responsibility` (`financial_responsibility`),
  KEY `party_sex` (`party_sex`),
  KEY `cellphone_use` (`cellphone_use`),
  KEY `movement_preceding_collision` (`movement_preceding_collision`),
  KEY `other_associate_factor_1` (`other_associate_factor_1`),
  KEY `other_associate_factor_2` (`other_ass

## Code to dump each csv row into the MySQL db

In [291]:
import pandas as pd

### Collision

In [474]:
dict_pcf_violation_category = {}

dict_location_type = {}
dict_lighting = {}
dict_road_condition = {}
dict_road_surface = {}
dict_weather = {}

dict_type_of_collision = {}
dict_collision_severity = {}
dict_hit_and_run = {}

In [475]:
def dump_pcfs(row):
    if(row.pcf_violation_category is None):
        pcf_viol_categ_id = None
    elif row.pcf_violation_category in dict_pcf_violation_category:
        pcf_viol_categ_id = dict_pcf_violation_category[row.pcf_violation_category]
    else:
        cursor.execute("""INSERT INTO PcfViolationCategory
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.pcf_violation_category,))
        cnx.commit()

        cursor.execute("SELECT id FROM PcfViolationCategory WHERE description=%s LIMIT 0, 1", (row.pcf_violation_category,))
        pcf_viol_categ_id = cursor.fetchone()[0]

        dict_pcf_violation_category[row.pcf_violation_category] = pcf_viol_categ_id

    cursor.execute("""INSERT INTO Pcfs
        VALUES (%s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""", (row.case_id, row.pcf_violation, pcf_viol_categ_id, row.pcf_violation_subsection))

In [476]:
def dump_locations(row):
    cursor.execute("""INSERT INTO Locations
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""", (row.case_id, row.population, row.county_city_location))

In [477]:
def dump_location_type(row):
    if(row.location_type is None):
        location_type_id = None
    elif row.location_type in dict_location_type:
        location_type_id = dict_location_type[row.location_type]
    else:
        cursor.execute("""INSERT INTO LocationType
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.location_type,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM LocationType WHERE description=%s LIMIT 0, 1", (row.location_type,))
        location_type_id = cursor.fetchone()[0]

        dict_location_type[row.location_type] = location_type_id

    return location_type_id

In [478]:
def dump_lighting(row):
    if(row.lighting is None):
        lighting_id = None
    elif row.lighting in dict_lighting:
        lighting_id = dict_lighting[row.lighting] 
    else:
        cursor.execute("""INSERT INTO Lighting
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.lighting,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM Lighting WHERE description=%s LIMIT 0, 1", (row.lighting,))
        lighting_id = cursor.fetchone()[0]

        dict_lighting[row.lighting] = lighting_id

    return lighting_id

In [479]:
def dump_road_condition_1(row):
    if(row.road_condition_1 is None):
        road_cond1_id = None
    elif row.road_condition_1 in dict_road_condition:
        road_cond1_id = dict_road_condition[row.road_condition_1] 
    else:
        cursor.execute("""INSERT INTO RoadCondition
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.road_condition_1,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM RoadCondition WHERE description=%s LIMIT 0, 1", (row.road_condition_1,))
        road_cond1_id = cursor.fetchone()[0]

        dict_road_condition[row.road_condition_1] = road_cond1_id

    return road_cond1_id

In [480]:
def dump_road_condition_2(row):
    if(row.road_condition_2 is None):
        road_cond2_id = None
    elif row.road_condition_2 in dict_road_condition:
        road_cond2_id = dict_road_condition[row.road_condition_2] 
    else:
        cursor.execute("""INSERT INTO RoadCondition
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.road_condition_2,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM RoadCondition WHERE description=%s LIMIT 0, 1", (row.road_condition_2,))
        road_cond2_id = cursor.fetchone()[0]

        dict_road_condition[row.road_condition_2] = road_cond2_id
    
    return road_cond2_id

In [481]:
def dump_road_surface(row):
    if(row.road_surface is None):
        road_surface_id = None
    elif row.road_surface in dict_road_surface:
        road_surface_id = dict_road_surface[row.road_surface] 
    else:
        cursor.execute("""INSERT INTO RoadSurface
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.road_surface,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM RoadSurface WHERE description=%s LIMIT 0, 1", (row.road_surface,))
        road_surface_id = cursor.fetchone()[0]

        dict_road_surface[row.road_surface] = road_surface_id

    return road_surface_id

In [482]:
def dump_weather_1(row):
    if(row.weather_1 is None):
        weather1_id = None
    elif row.weather_1 in dict_weather:
        weather1_id = dict_weather[row.weather_1] 
    else:
        cursor.execute("""INSERT INTO Weather
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.weather_1,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM Weather WHERE description=%s LIMIT 0, 1", (row.weather_1,))
        weather1_id = cursor.fetchone()[0]

        dict_weather[row.weather_1] = weather1_id
    
    return weather1_id

In [483]:
def dump_weather_2(row):
    if(row.weather_2 is None):
        weather2_id = None
    elif row.weather_2 in dict_weather:
        weather2_id = dict_weather[row.weather_2] 
    else:
        cursor.execute("""INSERT INTO Weather
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.weather_2,))
        cnx.commit()
        
        cursor.execute("SELECT id FROM Weather WHERE description=%s LIMIT 0, 1", (row.weather_2,))
        weather2_id = cursor.fetchone()[0]

        dict_weather[row.weather_2] = weather2_id
    
    return weather2_id

In [484]:
def dump_factors(row):
    location_type_id = dump_location_type(row)
    lighting_id = dump_lighting(row)
    road_cond1_id = dump_road_condition_1(row)
    road_cond2_id = dump_road_condition_2(row)
    road_surface_id = dump_road_surface(row)
    weather1_id = dump_weather_1(row)
    weather2_id = dump_weather_2(row)

    cursor.execute("""INSERT INTO Factors
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""",(row.case_id, location_type_id, lighting_id, road_cond1_id, road_cond2_id, road_surface_id, weather1_id, weather2_id))

In [485]:
def dump_cases(row):
    cursor.execute("""INSERT INTO Cases
        VALUES (%s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""",(row.case_id, row.process_date, row.officer_id, row.jurisdiction))

In [486]:
def dump_type_of_collision(row):
    if(row.type_of_collision is None):
        col_type_id = None
    elif row.type_of_collision in dict_type_of_collision:
        col_type_id = dict_type_of_collision[row.type_of_collision] 
    else:
        cursor.execute("""INSERT INTO TypeOfCollision 
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.type_of_collision,))
        cnx.commit()
        cursor.execute("SELECT id FROM TypeOfCollision WHERE description=%s LIMIT 0, 1", (row.type_of_collision,))
        col_type_id = cursor.fetchone()[0]

        dict_type_of_collision[row.type_of_collision] = col_type_id

    return col_type_id

In [487]:
def dump_collision_severity(row):
    if row.collision_severity in dict_collision_severity:
        col_severity_id = dict_collision_severity[row.collision_severity] 
    else:
        cursor.execute("""INSERT INTO CollisionSeverity
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.collision_severity,))
        cnx.commit()
        cursor.execute("SELECT id FROM CollisionSeverity WHERE description=%s LIMIT 0, 1", (row.collision_severity,))
        col_severity_id = cursor.fetchone()[0]

        dict_collision_severity[row.collision_severity] = col_severity_id

    return col_severity_id

In [488]:
def dump_hit_and_run(row):
    if row.hit_and_run in dict_hit_and_run:
        hit_n_run_id = dict_hit_and_run[row.hit_and_run] 
    else:
        cursor.execute("""INSERT INTO HitAndRun
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (row.hit_and_run,))
        cnx.commit()
        cursor.execute("SELECT id FROM HitAndRun WHERE description=%s LIMIT 0, 1", (row.hit_and_run,))
        hit_n_run_id = cursor.fetchone()[0]

        dict_hit_and_run[row.hit_and_run] = hit_n_run_id

    return hit_n_run_id

In [489]:
def dump_collision_row(row):
    col_type_id = dump_type_of_collision(row)
    col_severity_id = dump_collision_severity(row)
    hit_n_run_id = dump_hit_and_run(row)

    cursor.execute("""INSERT INTO Collisions
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""", (row.case_id, row.collision_date, row.collision_time, col_type_id, col_severity_id, hit_n_run_id, row.tow_away))
    cnx.commit()

    dump_pcfs(row)
    dump_locations(row)
    dump_factors(row)
    dump_cases(row)

In [490]:
def import_collisions(df):
    for idx, row in enumerate(df.itertuples(index=True, name="Pandas")):
        if(idx % 10000 == 0):
            print(f"row number {idx}")
        dump_collision_row(row)

In [208]:
collisions = pd.read_csv('../collisions2018.csv')

/Users/xavier/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [211]:
collisions = collisions.astype(object).where(pd.notnull(collisions),None)

In [491]:
import_collisions(collisions)

row number 0
row number 10000
row number 20000
row number 30000
row number 40000
row number 50000
row number 60000
row number 70000
row number 80000
row number 90000
row number 100000
row number 110000
row number 120000
row number 130000
row number 140000
row number 150000
row number 160000
row number 170000
row number 180000
row number 190000
row number 200000
row number 210000
row number 220000
row number 230000
row number 240000
row number 250000
row number 260000
row number 270000
row number 280000
row number 290000
row number 300000
row number 310000
row number 320000
row number 330000
row number 340000
row number 350000
row number 360000
row number 370000
row number 380000
row number 390000
row number 400000
row number 410000
row number 420000
row number 430000
row number 440000
row number 450000
row number 460000
row number 470000
row number 480000
row number 490000
row number 500000
row number 510000
row number 520000
row number 530000
row number 540000
row number 550000
row nu

In [188]:
cursor.execute("SHOW PROCESSLIST")
res = cursor.fetchall()
for x in res:
    print(x)

(4, 'event_scheduler', 'localhost', None, 'Daemon', 194392, 'Waiting on empty queue', None)
(9, 'root', 'localhost', None, 'Sleep', 23, '', None)
(10, 'root', '127.0.0.1:46402', None, 'Sleep', 12, '', None)
(24701, 'root', '127.0.0.1:32920', None, 'Sleep', 1, '', None)
(24751, 'root', '185.25.195.61:59564', 'data', 'Sleep', 289, '', None)
(24759, 'root', '127.0.0.1:33090', None, 'Sleep', 12, '', None)
(24774, 'root', '185.25.195.61:59661', 'data', 'Query', 0, 'starting', 'SHOW PROCESSLIST')
(24789, 'root', '127.0.0.1:33176', None, 'Sleep', 12, '', None)


## Parties

In [505]:
dicts = {}

In [564]:
def dump_sattelite(row, table_name, col_name, can_null=True, val_mapping={}):
    if col_name not in dicts:
        dicts[col_name] = {}

    if f"{row[col_name]}" in val_mapping:
        col_value = val_mapping[f"{row[col_name]}"]
    else:
        col_value = row[col_name]  

    if can_null and col_value is None:
        ret_id = None
    elif col_value in dicts[col_name]:
        ret_id = dicts[col_name][col_value] 
    else:
        cursor.execute(f"""INSERT INTO {table_name}
            (description)
            VALUES (%s)
            ON DUPLICATE KEY UPDATE id=id""", (col_value,))
        cnx.commit()
        cursor.execute(f"SELECT id FROM {table_name} WHERE description=%s LIMIT 0, 1", (col_value,))
        ret_id = cursor.fetchone()[0]

        dicts[col_name][col_value] = ret_id

    return ret_id

In [569]:
def dump_vehicles(row):
    statewide_vehicle_type_id = dump_sattelite(row, "StatewideVehiculeType", "statewide_vehicle_type")
    vehicle_make_id = dump_sattelite(row, "VehiculeMake", "vehicle_make")

    if row["school_bus_related"] is None:
        school_bus_related = None
    else: 
        school_bus_related = True

    cursor.execute("""INSERT INTO Vehicles
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE party_id=party_id""", (row["id"], school_bus_related, statewide_vehicle_type_id, vehicle_make_id, row["vehicle_year"]))

In [566]:
def dump_party_row(row):
    financial_resp_id = dump_sattelite(row, "FinancialResponsibility", "financial_responsibility")
    sex_id = dump_sattelite(row, "PersonSex", "party_sex")
    cellphone_use_id = dump_sattelite(row, "CellphoneUse", "cellphone_use", val_mapping={"1": "B", "2": "C", "3": "D"})
    movmt_preceding_col_id = dump_sattelite(row, "MovementPrecedingCollision", "movement_preceding_collision")
    other_assoc_fac_1_id = dump_sattelite(row, "OtherAssociatedFactors", "other_associate_factor_1")
    other_assoc_fac_2_id = dump_sattelite(row, "OtherAssociatedFactors", "other_associate_factor_2")
    party_drug_phys_id = dump_sattelite(row, "PartyDrugPhysical", "party_drug_physical", val_mapping={"G": None})
    party_safety_equip_1_id = dump_sattelite(row, "PartySafetyEquipement", "party_safety_equipment_1")
    party_safety_equip_2_id = dump_sattelite(row, "PartySafetyEquipement", "party_safety_equipment_2")
    party_sobriety_id = dump_sattelite(row, "PartySobriety", "party_sobriety")

    cursor.execute("""INSERT INTO Parties
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""", (row["id"], row["case_id"], row["party_number"], financial_resp_id, row["party_age"], sex_id, row["at_fault"], cellphone_use_id, row["hazardous_materials"], movmt_preceding_col_id, other_assoc_fac_1_id, other_assoc_fac_2_id, party_drug_phys_id, party_safety_equip_1_id, party_safety_equip_2_id, party_sobriety_id))

    dump_vehicles(row)

In [567]:
def import_parties(df):
    for idx, (k, row) in enumerate(df.iterrows()):
        if(idx % 10000 == 0):
            print(f"row number {idx}")
        dump_party_row(row)

In [510]:
parties = pd.read_csv('../parties2018.csv')

/Users/xavier/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,13,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [511]:
parties = parties.astype(object).where(pd.notnull(parties),None)

In [570]:
import_parties(parties)

row number 0
row number 10000
row number 20000
row number 30000
row number 40000
row number 50000
row number 60000
row number 70000
row number 80000
row number 90000
row number 100000
row number 110000
row number 120000
row number 130000
row number 140000
row number 150000
row number 160000
row number 170000
row number 180000
row number 190000
row number 200000
row number 210000
row number 220000
row number 230000
row number 240000
row number 250000
row number 260000
row number 270000
row number 280000
row number 290000
row number 300000
row number 310000
row number 320000
row number 330000
row number 340000
row number 350000
row number 360000
row number 370000
row number 380000
row number 390000
row number 400000
row number 410000
row number 420000
row number 430000
row number 440000
row number 450000
row number 460000
row number 470000
row number 480000
row number 490000
row number 500000
row number 510000
row number 520000
row number 530000
row number 540000
row number 550000
row nu

## Victims

In [580]:
def dump_victim_row(row):
    victim_sex_id = dump_sattelite(row, "PersonSex", "victim_sex")
    victim_safety_equip_1_id = dump_sattelite(row, "VictimSafetyEquipment", "victim_safety_equipment_1")
    victim_safety_equip_2_id = dump_sattelite(row, "VictimSafetyEquipment", "victim_safety_equipment_2")
    victim_deg_injury_id = dump_sattelite(row, "VictimDegreeOfInjury", "victim_degree_of_injury", val_mapping={"7": None})

    cursor.execute("""INSERT INTO Victims
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE case_id=case_id""", (row["id"], row["case_id"], row["party_number"], row["victim_age"], victim_sex_id, victim_deg_injury_id, row["victim_ejected"], row["victim_role"], victim_safety_equip_1_id, victim_safety_equip_2_id, row["victim_seating_position"]))

In [572]:
def import_victims(df):
    for idx, (k, row) in enumerate(df.iterrows()):
        if(idx % 10000 == 0):
            print(f"row number {idx}")
        dump_victim_row(row)

In [573]:
victims = pd.read_csv('../victims2018.csv')

/Users/xavier/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [574]:
victims = victims.astype(object).where(pd.notnull(victims),None)

In [581]:
import_victims(victims)

row number 0
row number 10000
row number 20000
row number 30000
row number 40000
row number 50000
row number 60000
row number 70000
row number 80000
row number 90000
row number 100000
row number 110000
row number 120000
row number 130000
row number 140000
row number 150000
row number 160000
row number 170000
row number 180000
row number 190000
row number 200000
row number 210000
row number 220000
row number 230000
row number 240000
row number 250000
row number 260000
row number 270000
row number 280000
row number 290000
row number 300000
row number 310000
row number 320000
row number 330000
row number 340000
row number 350000
row number 360000
row number 370000
row number 380000
row number 390000
row number 400000
row number 410000
row number 420000
row number 430000
row number 440000
row number 450000
row number 460000
row number 470000
row number 480000
row number 490000
row number 500000
row number 510000
row number 520000
row number 530000
row number 540000
row number 550000
row nu

# Data correction

`Collisions` is not as clean as I thought

This is to clean the `hit_and_run` with value `D`

In [597]:
# cursor.execute("SELECT * FROM Victims WHERE case_id='2816618'")
# res = cursor.fetchall()
# for x in res:
#     print(x)

# case_id 2816618

cursor.execute("DELETE FROM Victims WHERE case_id='2816618'")

cursor.execute("DELETE FROM Vehicles WHERE party_id='5234392'")
cursor.execute("DELETE FROM Vehicles WHERE party_id='5234393'")
cursor.execute("DELETE FROM Parties WHERE case_id='2816618'")

cursor.execute("DELETE FROM Locations WHERE case_id='2816618'")
cursor.execute("DELETE FROM Pcfs WHERE case_id='2816618'")
cursor.execute("DELETE FROM Factors WHERE case_id='2816618'")
cursor.execute("DELETE FROM Cases WHERE case_id='2816618'")

cursor.execute("DELETE FROM Collisions WHERE case_id='2816618'")

This is to clean the `pcf_violation_category` with value `21804`. 

As the column can be `NULL`, we simply delete the line. 

The same collision was responsible for all the anomalies hence we can just delete the corresponding `PcfViolationCategory` and that's it.

In [605]:
# cursor.execute("SELECT * FROM Pcfs WHERE pcf_violation_category in (SELECT id FROM PcfViolationCategory WHERE description='21804')")
cursor.execute("DELETE FROM PcfViolationCategory WHERE description='21804'")

This is to clean the `road_surface` with value `H`.

As the column can be `NULL`, we simply delete the line. 

The same collision was responsible for all the anomalies hence we can just delete the corresponding `RoadSurface` and that's it.

In [609]:
# cursor.execute("SELECT * FROM Factors WHERE road_surface in (SELECT id FROM RoadSurface WHERE description='H')")
cursor.execute("DELETE FROM RoadSurface WHERE description='H'")